#**<font size="10" color='cornflowerblue'><ins>Welcome to the end-term project**
## **Note**: After choosing or changing an answer, run its block to save changes.
## **Note**: It is best to avoid Runtime -> Run all.<br><br>


#**Important**: Every multiple-choice question in the project has one possible answer. You are asked to choose the <ins>most accurate answer</ins> among all options (התשובה הנכונה ביותר)


Block 1#


In [ ]:
#@title #**What is your ID and group number? (<font color="red">*Mandatory</font>)**


def initialize_colab():
  # Verify credentials
  if id_number < 1000 or group_number == 0 or group_number > 1000:
    raise Exception("Are you sure these details are correct?") 
  # Messages to user
  print("ID number saved: ", id_number)
  print("Group number saved: ", group_number)
  print("\nYou may now begin, good luck!")

def generate_questions():
  Qs_per_pool = {1:2,2:1,3:1,4:1,5:1}
  Q1 = ['1_1','1_2','1_3','1_4','1_5']
  Q2 = ['2_1','2_3']
  Q3 = ['3_1','3_2']
  Q4 = ['4_1','4_4']
  Q5 = ['4_2','4_3']
  Qlist = [] # add lists of [Qnumber,version]
  for i in range(len(Qs_per_pool.keys())): #choose questions from each pool
    np.random.seed(10*group_number+i)
    add_q = list(zip(np.random.choice(eval(f"Q{i+1}"),2, replace=False),np.random.choice(['V1','V2'],2)))
    for k in range(Qs_per_pool[i+1]):
      Qlist.append(add_q[k])
  data['Qlist'] = Qlist
  return Qlist

try:
  # Imports and library initialization
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import numpy as np
  import matplotlib.pyplot as plt
  import seaborn as sns
  import pandas as pd
  from math import sqrt
  from scipy import stats
  import statsmodels.api as sm
  from statsmodels.formula.api import ols
  from sklearn import linear_model
  from ipywidgets import interact, fixed
  from IPython.display import display, HTML
  from tabulate import tabulate
  from datetime import datetime
  sns.set(style = "white", palette = "muted", color_codes = True)
  sns.set(rc = {'figure.figsize': (10, 10)});
  sns.set_context("talk")

  # Setting up global colab variables
  exercise_name = "finalproj"
  version = "r14"
  executed_blocks = {0, 1}
  blocks_mapping = {6: '1.1', 7: '1.2', 9: '2.1', 10: '2.2', 12: '3.1', 
                    13: '3.2', 15: '4.1', 16: '4.2', 17: '4.3', 18: '4.4',
                    20: '5', 22: '6', 24: '7', 27: '8', 30: '9', 
                    33: '10', 36: '11', 38: '12', 40: '13'}
  answers = {}
  data = {'version': version}
  id_number =  0#@param {type:"integer"}
  group_number =  0#@param {type:"integer"}
  initialize_colab()
  Qlist = generate_questions()
  q7_flag = False # Signals if question 7's block was already run
  initialized = True
except Exception as e:
  print(str(e))


# Dependency check function
def verify_dependencies(block_id, required_blocks_indices=[]):
  for block_index in required_blocks_indices:
    if block_index not in executed_blocks:
      raise Exception("You must first run block "+str(block_index)+"#")


# **********************  Statistics helper functions  **********************
 
def boot_dist_mean_diff(dist1,dist2, B=10000,replacement = True):
  sample_matrix1 = np.random.choice(dist1,size=(len(dist1),B),replace=replacement)
  means_dist1 = np.mean(sample_matrix1,axis=0) # average each column resulting in len(dist) X 1 vector
  sample_matrix2 = np.random.choice(dist2,size=(len(dist2),B),replace=replacement)
  means_dist2 = np.mean(sample_matrix2,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist1-means_dist2)

def perm_dist_mean_diff(dist1,dist2, B=10000,replacement = True):
  sample_matrix1 = np.random.choice(np.concatenate([dist1,dist2]),size=(len(dist1),B),replace=replacement)
  means_dist1 = np.mean(sample_matrix1,axis=0) # average each column resulting in len(dist) X 1 vector
  sample_matrix2 = np.random.choice(np.concatenate([dist1,dist2]),size=(len(dist2),B),replace=replacement)
  means_dist2 = np.mean(sample_matrix2,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist1-means_dist2)

def t_fun(deg_freedom,deg_freedom_extra,t_score):
  pval = (stats.t.sf((t_score), deg_freedom))  # two-sided pvalue = Prob(abs(t)>tt)
  if (t_score<0):
    pval=1-pval #
  print ("t(",deg_freedom ,") = ", t_score,", p-value:", round(pval, 5))
  ttt = stats.t.rvs(deg_freedom,size=5000)
  plot_dist(ttt, height=5,bins=20,color="orange", fit="t")
  plt.axvline(t_score,color="black",linestyle="--",label="Chosen value")
  plt.title("t-distribution")


def F_fun(df1,df2,F_score):
  df1 = float(df1) ##k-1
  df2 =float(df2) ##N-k
  F_score = float(F_score)  ##MSB/MSW
  pval = (stats.f.sf(F_score, df1,df2))  # two-sided pvalue = Prob(abs(t)>tt)
  print("F(",df1,",",df2 ,") = ", F_score,", p-value:", round(pval,3),"\n")
  fff = stats.f.rvs(df1,df2,size=1000)
  sns.distplot(fff,hist=1,kde=0,axlabel="F-score",fit=stats.f,fit_kws={"color":"purple"})
  plt.xlim(0,max(10,F_score*1.1))
  plt.axvline(F_score,color="black",linestyle="--")
  plt.title("F-distribution")


def plot_dist(dist, bins="auto", kde=False, fit="None", color="b", stat="density",height=8, aspect=1):
  plt.figure(0);
  sns.displot(dist, kde=kde,bins=bins,stat=stat,color=color,height=height, aspect=aspect);
  if (fit=="Normal") & (stat=="density"):  
    d = stats.norm.fit(dist)
    mu = d[0]; sigma = d[1]
    x = np.linspace(mu - 3.5*sigma, mu + 3.5*sigma, 200) #limits of dist
    plt.plot(x, stats.norm.pdf(x, mu, sigma),color="black");
  elif (fit=="F") & (stat=="density"):  
    d = stats.f.fit(dist)
    dfn = d[0]; dfd = d[1]
    interval = stats.f.interval(.995,dfn,dfd) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.f.pdf(x, dfn, dfd),color="black");
  elif (fit=="gamma") & (stat=="density"):  
    d = stats.gamma.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.gamma.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.gamma.pdf(x, a, loc, scale),color="black");
  elif (fit=="t") & (stat=="density"):  
    d = stats.t.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.t.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.t.pdf(x, a, loc, scale),color="black");


def perform_fdr_t(t_vals, deg_freedom, alpha):
  old_t=t_vals.copy()
  t_vals=np.sort(t_vals)[::-1]
  pvals = [(stats.t.sf(np.abs(t_score), deg_freedom)) for t_score in t_vals]
  sig=0
  for i in range(len(pvals)):
    if pvals[i]<(alpha*(i+1)/len(t_vals)):
      sig=pvals[i]
  return (sig)


def boot_dist(dist, B=10000,replacement = True):
  sample_matrix = np.random.choice(dist,size=(len(dist),B),replace=replacement)
  means_dist = np.mean(sample_matrix,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist)


def boot_dist_F(data,IV,DV, B=10000, replacement = True):
  levels = np.unique(data[IV])
  F_vals=[]
  for i in range(B):
    samples = [np.random.choice(data[data[IV]==level][DV],size=len(data[data[IV]==level][DV]),
                                replace=replacement) for level in levels]
    f,_=stats.f_oneway(*samples) # average each column resulting in len(dist) X 1 vector
    F_vals.append(f) 
  return (F_vals)


def cohen_d(x,y):
  nx = len(x)
  ny = len(y)
  dof = nx + ny - 2
  return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


def plot_reg_results(x,y):
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
  term="+"*(intercept>0)
  if p_value<0.001: p_value=0.001
  pred_y = r"$\tildey$"
  reg_label = (f"{pred_y}={str(np.round(slope,1))}x{term}{str(np.round(intercept,1))}, p<{np.round(p_value,3)}")
  plt.figure(figsize=(7,7));
  sns.regplot(x=x,y=y);
  plt.title(reg_label);
  return ([slope,intercept,p_value])


def r_score(x, y):
    return round(stats.pearsonr(x, y)[0],3)


def plot_reg_and_minimize(betax,beta0,X,Y):
  student_x = X
  student_y = betax*student_x+beta0
  plt.figure(figsize=(12,8))
  sns.regplot( x=X,y=Y,fit_reg=False, color = "#154318")
  sns.lineplot(x=student_x, y=student_y, color = "r",label=("y="+str(betax)+"x+"+str(beta0)) )
  plt.xlabel('x', color='#1C2833')
  plt.ylabel('y', color='#1C2833')
  plt.ylim(min(Y)*.95,max(Y)*1.05)
  plt.xlim(min(X)*.95,max(X)*1.05)
  SSerror = np.round(np.sum((Y-student_y)**2))
  SST=np.round(np.sum((Y-Y.mean())**2))
  plt.legend(loc='upper left')
  plt.title(f"Your graph:\nSSE ={str(int(SSerror))}, SST={SST}")
  slope = (r_score(X,Y)*np.std(Y)/np.std(X))


def spearman_score(x, y):
    return round(stats.spearmanr(x, y)[0],3)


# **********************  Other helper functions  **********************
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '20px'
      rule.style.color = 'darkred'
      break
    }
  }
  '''))

def reset_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      print(rule.style.fontSize)
      print(rule.style.color)
      rule.style.fontSize = '14px'
      rule.style.color = 'black'
      break
    }
  }
  '''))


# **********************  Helper class  **********************
class Polynomial(list): 
    def __repr__(self):
        # joiner[first, negative] = str
        joiner = {
            (True, True): '-',
            (True, False): '',
            (False, True): ' - ',
            (False, False): ' + '
        }
 
        result = []
        for power, coeff in reversed(list(enumerate(self))):
            j = joiner[not result, coeff < 0]
            coeff = abs(coeff)
            if coeff == 1 and power != 0:
                coeff = ''
 
            f = {0: '{}{}', 1: '{}{}x'}.get(power, '{}{}x^{}')
 
            result.append(f.format(j, coeff, power))
 
        return ''.join(result) or '0'


# ********************** Project functions: Part 1 **********************
def Chi_squared_fun(deg_freedom,dfextra,chisquare):
    deg_freedom=float(deg_freedom); chisquare=float(chisquare)
    pval = (stats.chi2.sf(chisquare, deg_freedom))  # two-sided pvalue = Prob(abs(t)>tt)
    print ("χ2(",deg_freedom ,") = ", chisquare,", p-value:", pval)
    dist = stats.chi2.rvs(deg_freedom,size=5000)
    plt.figure(figsize=(10,6))
    sns.displot(dist,kde=1)
    plt.axvline(chisquare,color="black",linestyle="--",label="Chosen value")
    plt.title(f"χ2({deg_freedom}) distribution")

def perform_chisquared(exp,obs):
  exp = np.array(exp); obs=np.array(obs)
  if len(exp)!=len(obs):
    print("please enter observed and expected groups in the same size")
    return
  return (sum((exp-obs)**2/exp))

def perform_anova_twoway(data, dv, iv1,iv2,print_out=True,effect=0):
  if (iv1=="leagueCountry")&(iv2=="position"):
    iv1="position"
    iv2="leagueCountry"
  if (dv=="")|(iv1=="")|(iv2==""):
    print ("Please set all required variables")
    return
  try:
    model = ols(f'{dv} ~ {iv1}*{iv2}', data=data).fit()
  except:
    return
  res = sm.stats.anova_lm(model, typ=1).round(4)
  results_print = res.drop(["df"],axis=1)
  results_print = results_print.drop(["PR(>F)"],axis=1)
  if print_out:
    display(HTML(results_print.to_html()))
  return (res,res['F'][effect],(res['df'][effect],res['df'][3])) #set F index to fit the experimental question

def perform_anova_oneway(data, dv, iv1,iv2=None,print_out=True,effect=""):
  if (dv=="")|(iv1==""):
    print ("Please set all required variables")
  try:
    model = ols(f'{dv} ~ {iv1}', data=data).fit()
  except:
    print("An error occured. please check the spelling of factors")
    return
  res = sm.stats.anova_lm(model, typ=1).round(4)
  results_print = res.drop(["df"],axis=1)
  results_print = results_print.drop(["PR(>F)"],axis=1)
  if print_out:
    display(HTML(results_print.to_html()))
  return (res,res['F'][0],(res['df'][0],res['df'][1]))

def perform_t_test(data, dv, iv, levels,print_out=True,effect=""):
  if (dv=="")|(iv=="")|(levels==""):
    print ("Please set all required variables")
    return
  try:
    levels=(levels.split(","))#eval(f"[{levels}]")
  except:
    print("An error occured. did you enter the levels correctly with a , between them?")
    return  
  try:
    test = stats.ttest_ind(data[data[iv]==levels[0]][dv],
                           data[data[iv]==levels[1]][dv])
  except:
    print("An error occured. please check the spelling of factors")
    return
  test=np.round(test,4)
  res_print= f"independent t-test between the {dv} of {levels[0]} and {levels[1]}:\n t={test[0]}"
  if print_out:
    print(res_print)
  dof=len(data[data[iv]==levels[0]])+len(data[data[iv]==levels[1]])-2
  return (test,test[0],(dof,0))

def perform_cor_test(data,y,x,add="",print_out=True,effect=""):
  if (x=="")|(y==""):
    print ("Please set all required variables")
    return
  try:
    test = stats.pearsonr(data[x],data[y])
  except:
    print("An error occured. please check the spelling of factors")
    return
  test=np.round(test,4)
  df = len(data[x])-2
  tscore=np.round(test[0]/np.sqrt((1-test[0]**2)/df),2)

  res_print= f"Pearson correlation between the {x} and {y}:\n r={test[0]}, t={tscore}"
  if print_out:
    print(res_print)
  return (test,tscore,(df,0))

def perform_spearman_test(data,y,x,add="",print_out=True,effect=""):
  if (x=="")|(y==""):
    print ("Please set all required variables")
    return
  try:
    test = stats.spearmanr(data[x],data[y])
  except:
    print("An error occured. please check the spelling of factors")
    return
  test=np.round(test,4)
  df = len(data[x])-2
  tscore=np.round(test[0]/np.sqrt((1-test[0]**2)/df),2)
  res_print= f"Spearman correlation between the {x} and {y}:\n r={test[0]}, t={tscore}"
  if print_out:
    print(res_print)
  return (test,tscore,(df,0))

def perform_linear_regression(data,y,x,add="",print_out=True,effect=""):
  if (x=="")|(y==""):
    print ("Please set all required variables")
    return
  try:
    test = stats.linregress(x=data[x],y=data[y])
  except:
    print("An error occured. please check the spelling of factors")
    return
  test=np.round(test,4)
  df = len(data[x])-2
  SSE = np.round(np.sum((test[0]*data[x]+test[1]-data[y])**2),2)
  SST = np.round(np.sum((data[y]-data[y].mean())**2),2)
  res_print= f"Linear regression to predict {y} by {x}:\n y={test[0]}x+{test[1]},\n SSE = {SSE},SST = {SST} ,p-value of slope={(test[3])}"
  if print_out: 
    print(res_print)
  return ([test,SSE,SST],test[0],(df,0))

def show_contingency_table(data,var1,var2):
  if (var1=="")|(var2==""):
    print ("Please set all required variables")
    return
  try:
    observed = (pd.crosstab(data[var1], data[var2]))
    print("Observed:")
    display(HTML(observed.to_html()))
  except:
    print("An error occured. please check the spelling of factors")
    return
  print(res_print)

def show_contingency_table_chi2(data,add,var1,var2):
  if (var1=="")|(var2==""):
    print ("Please set all required variables")
    return
  try:
    observed = (pd.crosstab(data[var1], data[var2]))
    print("Observed:")
    display(HTML(observed.to_html()))
    chi2, p, dof, expected = stats.chi2_contingency(observed.values,correction=False)  
    expected=pd.DataFrame(expected, columns=observed.columns, index=observed.index)
    print("Expected:")
    display(HTML(expected.to_html()))
    chi2=np.round(chi2,3)
  except:
    print("An error occured. please check the spelling of factors")
    return
  res_print= f"Chi-squared of independence between {var1} and {var2}:\n Chi-squared={chi2}"
  print(res_print)
  return (chi2,chi2,(dof,0))

def show_frequency_table(data,var):
  if (var==""):
    print ("Please set all required variables")
    return
  try:
    observed = data[var].value_counts()
    print("***** Frequencies *****\n")
    toprint=(pd.DataFrame((dict(observed)),index=[1]))
    display(HTML(toprint.to_html()))

  except Exception as e:
    print("An error occured. please check the spelling of factors")
    print(e)
    return

def show_frequency_table_chi2(data,add,var,add1=""):
  if (var==""):
    print ("Please set all required variables")
    return
  try:
    observed = data[var].value_counts()
    dof=len(observed)-1
    toprint=pd.DataFrame((dict(observed)),index=[1])
    print(observed)
    display(HTML(toprint.to_html()))
    expected = np.repeat(np.mean(observed),dof+1)
    expected_df = pd.DataFrame(np.transpose(expected), columns=toprint.index, index=toprint.columns)
    print(expected)
    display(HTML(expected_df.transpose().to_html()))

    chi2 = np.round(stats.chisquare(np.array(observed),expected)[0],3)
  except:
     print("An error occured. please check the spelling of factors")
     return
  res_print= f"Chi-squared test of the hypothesis that players are equally distributed between the levels of '{var}' variable:\n Chi-squared={chi2}"
  print(res_print)
  return (chi2,chi2,(dof,0))

def show_histogram(data,var):
  if (var==""):
    print ("Please set all required variables")
    return
  try:
    sns.displot(data=data,x=var,kde=1)
  except:
     print("An error occured. please check the spelling of factors")
     return


def show_scatterplot(data,x,y):
  if (x=="")|(y==""):
    print ("Please set all required variables")
    return
  try:
    sns.scatterplot(data=data,x=x,y=y, alpha=0.2)
    plt.title("A scatterplot of the data:\n Darker point indicates more than one observation in a location")
  except:
    print("An error occured. please check the spelling of factors")
    return

def create_dist_by_choice(data,var1,function, repetitions, replacement, shuffling_var1, shuffling_var2, var2,dv,effect="Interaction"):
  eff_dict={ "Set only in two-way ANOVA":None,"Interaction":2,"Factor 1":0,"Factor 2":1}
  #check that all variables expect shuffling_var2 and var2 ar changed from default
  if ((function==perform_anova_twoway) and (var2=="")) | (var1=="") | (dv==""):
    print ("Please set all required variables")
    return
  statistic = np.repeat(0,repetitions).astype(float)
  for i in range(repetitions): #Avner - if you have an idea how to split this for loop and run in parallel or anything to make this faster, it would be great
    np.random.seed(i)
    data_sampled = data.sample(len(data[dv]),replace=replacement,random_state=i)
    if shuffling_var1:
      data_sampled[var1]=np.random.choice(data_sampled[var1],size=len(data_sampled[var1]),replace=False)
    if (shuffling_var2 and (var2!="")):
      data_sampled[var2]=np.random.choice(data_sampled[var2],size=len(data_sampled[var2]),replace=False)
    curr_stat=function(data_sampled,dv,var1,var2,print_out=False,effect=eff_dict[effect])[1]
    statistic[i] = curr_stat
  sns.displot(x=statistic,bins=50)
  return statistic


def print_means_sd_by_factor(data,factor,numeric):
  plt.figure(figsize=(7,7))
  plt.hist(data[numeric])
  plt.title(f"Histogram of {numeric}")
  show_frequency_table(data,factor)
  print(f"\n\n***** Summary statistics of {numeric} *****\n")
  display(data[numeric].describe())

  df_means = data.groupby(factor)[numeric].mean()
  df_std = data.groupby(factor)[numeric].std()
  print(f"\n\n***** Summary statistics of {numeric} by {factor} *****\n")
  display(pd.DataFrame({"mean":df_means.values,"sd":df_std.values},index=df_means.index))
  print("\n\n")



# ********************** Project functions: Part 2 **********************
def Q1_1(V):
  np.random.seed(group_number)
  x= np.random.normal(50,5,50)
  y = 1.2*x + np.random.normal(0,9,50)
  possible_conc_qs = {'V1':"Increasing the consumption of Serotonin causes an increase in depression.",
                      'V2':"More Serotonin consumption is related to higher depression levels."}
  q_data = [x,y]
  q_head = "A researcher was interested in a possible cause of depression amongst adults. He measured the amount of Serotonin-related ingredients that 50 participants consumed in a single day, in grams. The researcher then measured each participant’s depression level on a scale between 0 to 100. Finally, he calculated the linear regression between the two variables."
  q_conc = possible_conc_qs[V]
  return(q_data,q_head,q_conc)


def Q1_2(V):
  np.random.seed(group_number+30)
  x= np.random.normal(50,5,50)
  y = 1.2*x + np.random.normal(0,9,50)
  prediction_model = lambda pred : np.round(pred*stats.linregress(x,y)[0]+stats.linregress(x,y)[1],2)
  possible_conc_qs = {'V1':f"A participant that consumed {np.round(np.min(x)-10,3)} grams of Serotonin-related ingredients is expected to have a depression level of {prediction_model(np.min(x)-10)}.",
                      'V2':f"A participant that consumed {np.round(np.mean(x)*0.95,3)} grams of Serotonin-related ingredients is expected to have a depression level of {prediction_model(np.mean(x)*0.95)}."}
  q_data = [x,y]
  q_head = "A researcher was interested in a possible cause of depression amongst adults. He measured the amount of Serotonin-related ingredients that 50 participants consumed in a single day, in grams. The researcher then measured each participant’s depression level on a scale between 0 to 100. Finally, he calculated the linear regression between the two variables."
  q_conc = possible_conc_qs[V]
  return(q_data,q_head,q_conc)


def Q1_3(V):
  np.random.seed(group_number+10)
  x= np.random.normal(50,5,50)
  y = 1.2*x + np.random.normal(0,9,50)
  if V=='V2':
    x[1]=150; y[1]=-150
  q_data = [x,y]
  q_head = "A researcher was interested in a possible cause of depression amongst adults. He measured the amount of Serotonin-related ingredients that 50 participants consumed in a single day, in grams. The researcher then measured each participant’s depression level on a scale between 0 to 100."
  q_conc = "There is a strong negative linear correlation between Serotonin-related ingredients and depression levels."
  
  return(q_data,q_head,q_conc)


def Q1_4(V):
  np.random.seed(1)
  x= np.random.normal(50,5,50)
  y = 1.2*x + np.random.normal(0,9,50)
  if V=='V2':
    np.random.seed(3)
    x= np.random.normal(80,3,400)
    y = 0.35*x + np.random.normal(15,13.5,400)
  q_data = [x,y]
  q_head = "A researcher was interested in a possible cause of depression amongst adults. He measured the amount of Serotonin-related ingredients that 50 participants consumed in a single day, in grams. The researcher then measured each participant’s depression level on a scale between 0 to 100."
  q_conc = "The relation between Serotonin-related ingredients and depression is meaningful and noteworthy."
  return(q_data,q_head,q_conc)

def Q1_5(V):
  np.random.seed(1)
  x= np.random.normal(50,5,300)
  y = 1.2*x + np.random.normal(0,4,300)
  if V=='V2':
    np.random.seed(3)
    x= np.random.normal(50,5,50)
    y = 0.01*x+50
  q_data = [x,y]
  q_head = "A researcher was interested in a possible cause of depression amongst adults. He measured the amount of Serotonin-related ingredients that 50 participants consumed in a single day, in grams. The researcher then measured each participant’s depression level on a scale between 0 to 100."
  q_conc = "The relation between Serotonin-related ingredients and depression is meaningful and noteworthy." 
  return(q_data,q_head,q_conc)


def Q2_1(V):
  np.random.seed(group_number+20)
  x1= np.random.normal(100,7,50)
  x2= np.random.normal(110,7,50)
  q_data = [x1,x2]
  q_head = "A researcher was interested in the reasoning skills of individuals after taking an experimental drug. The hypothesis was that the drug enhances reasoning. The researcher sampled 100 participants and gave 50 of them the new drug, and the other 50 received a placebo pill. The researcher then measured the participants' reasoning skills using a test with scores between 50 and 150. Finally, the researcher sampled 10,000 bootstrap samples from the original sample (with replacement), calculated the difference of means in each sample, and created a distribution"
  q_conc = "The percentage of values bigger than 0 in the bootstrap distribution is the p-value"
  if V=="V2":
    q_conc = "The percentage of values smaller than 0 in the bootstrap distribution is the p-value"
  return(q_data,q_head,q_conc)

def Q2_1(V):
  np.random.seed(group_number+20)
  x1= np.random.normal(100,7,50)
  x2= np.random.normal(110,7,50)
  q_data = [x1,x2]
  q_head = "A researcher was interested in the reasoning skills of individuals after taking an experimental drug. The hypothesis was that the drug enhances reasoning. The researcher sampled 100 participants and gave 50 of them the new drug, and the other 50 received a placebo pill. The researcher then measured the participants' reasoning skills using a test with scores between 50 and 150. Finally, the researcher sampled 10,000 bootstrap samples from the original sample (with replacement), calculated the difference of means in each sample, and created a distribution"
  q_conc = "The percentage of values bigger than the real data value (the difference between the two samples) in the bootstrap distribution is the p-value"
  if V=="V2":
    q_conc = "The percentage of values smaller than the real data value (the difference between the two samples) in the bootstrap distribution is the p-value"
  return(q_data,q_head,q_conc)


def Q2_3(V):
  np.random.seed(group_number+20)
  x1= np.random.normal(100,7,50)
  x2= np.random.normal(110,7,50)
  q_data = [x1,x2]
  q_head = "A researcher was interested in the reasoning skills of individuals after taking an experimental drug. The hypothesis was that the drug has some effect on reasoning (either positive or negative). The researcher sampled 100 participants and gave 50 of them the new drug, and the other 50 received a placebo pill. The researcher then measured the participants'  reasoning skills using a test with scores between 50 and 150. Finally, the researcher shuffled each observation’s label, calculated the difference of means in the mixed data, and repeated the process 10,000 times. The 10,000 values were used to create a distribution."
  q_conc = "The mean of the distribution is zero, this hints that the null hypothesis (H0) is correct."
  if V=="V2":
    q_conc = "The mean of the distribution is zero, and this hints that the null hypothesis (H0) is incorrect."
  return(q_data,q_head,q_conc)


def Q3_1(V):
  np.random.seed(group_number)
  x=np.round(np.random.normal(10, 3, 100)); x[x<1]=1
  text_add = "customers"; 
  if V=="V2":
    text_add = "cashiers";
  q_data = x
  q_head = f"A researcher was interested in the length of queues in supermarkets. She went and asked 100 {text_add} to list the number of people in their queue. " 
  q_conc = f"The average queue length is {np.mean(x)}."
  return(q_data,q_head,q_conc)


def Q3_2(V):
  np.random.seed(group_number)
  x=np.round(np.random.uniform(5, 80, 75))
  text_add = "movie goers"; 
  if V=="V2":
    text_add = "cinema employees (there was one in each movie theatre)";
  text_add2 = "went to";
  if V=="V2":
    text_add2 = "screened";
  q_data = x
  q_head = f"A researcher was interested in the attendance of movie screenings just before the COVID-19 outbreak. He went and asked 75 {text_add} how many people were attending the screening at the last movie they {text_add2}." 
  q_conc = f"The average attendance is {np.mean(x)}."
  return(q_data,q_head,q_conc)


def Q4_1(V):
  np.random.seed(group_number)
  if V=="V1":
    x=np.random.normal(172, 10, 20)
    y=np.random.normal(0, 2,20); y=y-np.mean(y); 
  if V=="V2":
    x=np.random.normal(172, 10, 20)
    y=np.random.uniform(0, 5, 20); 
  if V=="V3":
    x=np.random.normal(172, 10, 20)
    y=np.random.uniform(-5, 0, 20);
  sns.scatterplot(x=x,y=y);
  plt.axhline(0,color="grey"); 
  plt.ylim(-6,6)
  plt.xlabel("height")
  plt.ylabel("observed-predicted difference")
  q_data = [x,y]
  q_head = f"A researcher was interested in the relation between the height and weight of students.\nHe computed the linear regression between the two variables and decided to draw the differences between his observations and the linear line." 
  question = "\033[1mIs the graph of the researcher probable?\033[0m"
  print(q_head)
  print("")
  print(question)
  return(q_data,q_head,question)


def Q4_2_cross_val(show_me,V):
  good_seed=[3,7,8,10,11,13,15]
  np.random.seed(group_number)
  
  seed_cv = np.random.choice(good_seed)
  np.random.seed(seed_cv)
  N=30
  train_n=15
  test_n=15
  average_salary = np.random.normal(10, 5,N)
  years_of_education = np.round(np.random.normal(12, 4,N))
  number_of_kids = np.round(np.random.normal(3, 1,N))
  noise=np.random.normal( 0, 1,N)
  house_price = .2*average_salary + .3*years_of_education + .4*number_of_kids + noise
  
  SST=np.sum((house_price-house_price.mean())**2)
  X_model1_full = pd.DataFrame({"average_salary":average_salary,
                    "years_of_education":years_of_education,
                    "number_of_kids":number_of_kids})
  X_model1_full = sm.add_constant(X_model1_full) ## let's add an intercept (beta_0) to our model
  model1_full = sm.OLS(house_price, X_model1_full).fit()
  model_1_full_equation = f"house price (in millions) = {np.round(model1_full.params.values[1],3)}*{model1_full.params.index[1]} + ({np.round(model1_full.params.values[2],3)})*{model1_full.params.index[2]} + ({np.round(model1_full.params.values[3],3)})*{model1_full.params.index[3]} -{np.abs(np.round(model1_full.params.values[0],3))}"
  SSE_model_1_full = np.sum((model1_full.predict(X_model1_full)-house_price)**2) 

  X_model2_full = sm.add_constant(X_model1_full['average_salary']) ## let's add an intercept (beta_0) to our model
  model2_full = sm.OLS(house_price, X_model2_full).fit()
  model_2_full_equation = f"house price (in millions) = {np.round(model2_full.params.values[1],3)}*{model2_full.params.index[1]} -{np.abs(np.round(model1_full.params.values[0],3))}"
  SSE_model_2_full = np.sum((model2_full.predict(X_model2_full)-house_price)**2) 

  #crossval
  SST_test=np.sum((house_price[train_n:N]-house_price[train_n:N].mean())**2)
  X_model1_train = X_model1_full[0:train_n]
  X_model1_test = X_model1_full[train_n:N]
  model1_train = sm.OLS(house_price[0:train_n], X_model1_train).fit()
  model_1_train_equation = f"house price (in millions) = {np.round(model1_train.params.values[1],3)}*{model1_train.params.index[1]} + ({np.round(model1_train.params.values[2],3)})*{model1_train.params.index[2]} + ({np.round(model1_train.params.values[3],3)})*{model1_train.params.index[3]} -{np.abs(np.round(model1_train.params.values[0],3))}"
  SSE_model_1_crossval = np.sum((model1_train.predict(X_model1_test)-house_price[train_n:N])**2) 

  X_model2_train = X_model2_full[0:train_n]
  X_model2_test = X_model2_full[train_n:N]
  model2_train = sm.OLS(house_price[0:train_n], X_model2_train).fit()
  model_2_train_equation = f"house price (in millions) = {np.round(model2_train.params.values[1],3)}*{model2_train.params.index[1]} -{np.abs(np.round(model2_train.params.values[0],3))}"
  SSE_model_2_crossval = np.sum((model2_train.predict(X_model2_test)-house_price[train_n:N])**2) 

  return_dict = {"Model equation and SSE of fitting model 1 on all the data":(model_1_full_equation,SSE_model_1_full,SST,model1_full.predict(X_model1_full),house_price),
                "Model equation and SSE of fitting model 2 on all the data":(model_2_full_equation,SSE_model_2_full,SST,model2_full.predict(X_model2_full),house_price),
                "Model equation of fitting model 1 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data":(model_1_train_equation,SSE_model_1_crossval,SST_test,model1_train.predict(X_model1_test),house_price[train_n:N]),
                "Model equation of fitting model 2 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data":(model_2_train_equation,SSE_model_2_crossval,SST_test,model2_train.predict(X_model2_test),house_price[train_n:N])}
  sns.scatterplot(return_dict[show_me][3],return_dict[show_me][4])
  plt.xlabel("Predicted value from fitted model")
  plt.ylabel("Actual value observed")
  results = return_dict[show_me]
  print(f"\nModel equation: {results[0]}")
  print(f"SSE = {np.round(results[1],2)}")
  #print(f"SST = {np.round(results[2],2)}\n")


def Q4_3_cross_val(show_me,V):
  good_seed=[3,7,8,10,11,13,15]
  np.random.seed(group_number)
  
  seed_cv = np.random.choice(good_seed)
  np.random.seed(seed_cv)
  N=30
  train_n=15
  test_n=15
  average_salary = np.random.normal(10, 5,N)
  years_of_education = np.round(np.random.normal(12, 4,N))
  number_of_kids = np.round(np.random.normal(3, 1,N))
  noise=np.random.normal( 0, 1,N)
  house_price = .1*average_salary + noise
  
  SST=np.sum((house_price-house_price.mean())**2)
  X_model1_full = pd.DataFrame({"average_salary":average_salary,
                    "years_of_education":years_of_education,
                    "number_of_kids":number_of_kids})
  X_model1_full = sm.add_constant(X_model1_full) ## let's add an intercept (beta_0) to our model
  model1_full = sm.OLS(house_price, X_model1_full).fit()
  model_1_full_equation = f"house price (in millions) = {np.round(model1_full.params.values[1],3)}*{model1_full.params.index[1]} + ({np.round(model1_full.params.values[2],3)})*{model1_full.params.index[2]} + ({np.round(model1_full.params.values[3],3)})*{model1_full.params.index[3]} -{np.abs(np.round(model1_full.params.values[0],3))}"
  SSE_model_1_full = np.sum((model1_full.predict(X_model1_full)-house_price)**2) 

  X_model2_full = sm.add_constant(X_model1_full['average_salary']) ## let's add an intercept (beta_0) to our model
  model2_full = sm.OLS(house_price, X_model2_full).fit()
  model_2_full_equation = f"house price (in millions) = {np.round(model2_full.params.values[1],3)}*{model2_full.params.index[1]} -{np.abs(np.round(model1_full.params.values[0],3))}"
  SSE_model_2_full = np.sum((model2_full.predict(X_model2_full)-house_price)**2) 

  #crossval
  SST_test=np.sum((house_price[train_n:N]-house_price[train_n:N].mean())**2)
  X_model1_train = X_model1_full[0:train_n]
  X_model1_test = X_model1_full[train_n:N]
  model1_train = sm.OLS(house_price[0:train_n], X_model1_train).fit()
  model_1_train_equation = f"house price (in millions) = {np.round(model1_train.params.values[1],3)}*{model1_train.params.index[1]} + ({np.round(model1_train.params.values[2],3)})*{model1_train.params.index[2]} + ({np.round(model1_train.params.values[3],3)})*{model1_train.params.index[3]} -{np.abs(np.round(model1_train.params.values[0],3))}"
  SSE_model_1_crossval = np.sum((model1_train.predict(X_model1_test)-house_price[train_n:N])**2) 

  X_model2_train = X_model2_full[0:train_n]
  X_model2_test = X_model2_full[train_n:N]
  model2_train = sm.OLS(house_price[0:train_n], X_model2_train).fit()
  model_2_train_equation = f"house price (in millions) = {np.round(model2_train.params.values[1],3)}*{model2_train.params.index[1]} -{np.abs(np.round(model2_train.params.values[0],3))}"
  SSE_model_2_crossval = np.sum((model2_train.predict(X_model2_test)-house_price[train_n:N])**2) 

  return_dict = {"Model equation and SSE of fitting model 1 on all the data":(model_1_full_equation,SSE_model_1_full,SST,model1_full.predict(X_model1_full),house_price),
                "Model equation and SSE of fitting model 2 on all the data":(model_2_full_equation,SSE_model_2_full,SST,model2_full.predict(X_model2_full),house_price),
                "Model equation of fitting model 1 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data":(model_1_train_equation,SSE_model_1_crossval,SST_test,model1_train.predict(X_model1_test),house_price[train_n:N]),
                "Model equation of fitting model 2 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data":(model_2_train_equation,SSE_model_2_crossval,SST_test,model2_train.predict(X_model2_test),house_price[train_n:N])}
  sns.scatterplot(return_dict[show_me][3],return_dict[show_me][4])
  plt.xlabel("Predicted value from fitted model")
  plt.ylabel("Actual value observed")
  results = return_dict[show_me]
  print(f"\nModel equation: {results[0]}")
  print(f"SSE = {np.round(results[1],2)}")
  #print(f"SST = {np.round(results[2],2)}\n")


def Q4_2(V):
  q_head = f"A researcher was interested in the relation between house prices and three potential parameters - average income of the family, the average number of years of education for the parents, and the number of kids in the family.\nShe compared 2 different models - Model 1 with average income, number of years of education, and number of kids in the family as predictors (variables), and model 2 with only average income as a predictor (variable)." 
  question = "\033[1mIs the conclusion correct?\033[0m \n(Use the tools below in order to get more information about the models and see how they fit the data)"
  q_conc = "The best model to predict house prices is model 1."
  if V=="V2":
    q_conc = "The best model to predict house prices is model 2."
  print(q_head)
  print("")
  print(question)
  print(f"Conclusion: \033[1m{q_conc}\033[0m")
  interact(Q4_2_cross_val,show_me=["Model equation and SSE of fitting model 1 on all the data",
                                    "Model equation and SSE of fitting model 2 on all the data",
                                    "Model equation of fitting model 1 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data",
                                    "Model equation of fitting model 2 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data"] ,V=fixed(V));
  
  return(q_conc)


def Q4_3(V):
  q_head = f"A researcher was interested in the relation between house prices and three potential parameters - average income of the family, the average number of years of education for the parents, and the number of kids in the family.\nShe compared 2 different models - Model 1 with average income, number of years of education, and number of kids in the family as predictors (variables), and model 2 with only average income as a predictor (variable)." 
  question = "\033[1mIs the conclusion correct?\033[0m \n(Use the tools below in order to get more information about the models and see how they fit the data)"
  q_conc = "The best model to predict house prices is model 1."
  if V=="V2":
    q_conc = "The best model to predict house prices is model 2."
  print(q_head)
  print("")
  print(question)
  print(f"Conclusion: \033[1m{q_conc}\033[0m")
  interact(Q4_3_cross_val,show_me=["Model equation and SSE of fitting model 1 on all the data",
                                    "Model equation and SSE of fitting model 2 on all the data",
                                    "Model equation of fitting model 1 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data",
                                    "Model equation of fitting model 2 on 50% of the data, and SSE of it's prediction on the rest of the 50% of the data"] ,V=fixed(V));
  
  return(q_conc)


def Q4_4(V):
  q_head = "A researcher was interested in inferring if students’ faculty of study influences their salary (in thousands). She approached 240 students, 80 from humanities, 80 from exact sciences, and 80 from social sciences.\nShe used a one-way ANOVA analysis. She was also interested if gender influences students’ salary and so used the same data to conduct a one-way ANOVA on students’ gender.\nShe corrected the alpha from 0.05 to 0.025 in both studies, using the Bonferroni correction. Both effects resulted in a non-significant p-value." 
  q_conc = "The researcher concluded that both gender and faculty of study do not influence students’ salary."
  question = "\033[1mIs the conclusion correct?\033[0m"
  print(q_head)
  print("")
  print(question)
  print(f"Conclusion: \033[1m{q_conc}\033[0m")
  interact(Q4_4_anova,show_me=["Chi-square for independence of gender from faculty",
                               "One-way anova for gender","One-way anova for faculty","Two-way anova for gender and facutly",
                               "t-test between social and exact sciences","t-test between humanities and exact sciences",
                               "t-test between humanities and social sciences","t-test between men and women",
                                "Summary statistics of salary by gender", "Summary statistics of salary by faculty"] ,V=fixed(V));
  return(True)


def Q4_4_anova(show_me,V):
  from tabulate import tabulate

  men_social_sci = 10 
  men_exact_sci = 10
  if V=='V2':
    men_exact_sci = 8
  men_humanities = 10
  if V=='V2':
    men_humanities = 12 
  women_social_sci = 10 
  women_exact_sci = 10
  if V=='V2':
    women_exact_sci = 12
  women_humanities = 10 
  if V=='V2':
    women_humanities = 8
  sample_size_n = 40 
  sd_of_all_groups = 1 
  sample_size=sample_size_n
  means = [women_social_sci,women_exact_sci,women_humanities,men_social_sci, men_exact_sci,men_humanities ]
  sds  = np.repeat(sd_of_all_groups,len(means))

  columns = ["women","men"]
  rows = ["Social science","exact_sci","Humanities"]
  ##collect data and binary vectors for defining groups

  Gender = np.concatenate([ np.repeat( columns[i]  ,sample_size*len(rows)) for i in range(len(columns))])
  Faculty = np.concatenate([ np.repeat( rows[i]  ,sample_size*len(columns)/2) for i in range(len(rows))])
  Faculty = np.concatenate([Faculty, Faculty])
  curr_seed=501+int(np.floor(group_number/10)) #6 different prechecked seeds 
  np.random.seed(curr_seed)
  amp = np.concatenate([np.random.normal(means[i], sds[i], sample_size) for i in range(len(means))])

  ##arrange data frame:
  data = {"Gender":Gender, "Faculty":Faculty,"Salary":amp}
  groupsDF = pd.DataFrame(data, 
                        columns = ["Gender","Faculty","Salary"])
  
  sampled_Tmean = np.mean(data["Salary"])
  sampled_meansRows = [np.mean(data["Salary"][data["Faculty"]=="Social science"]),
                      np.mean(data["Salary"][data["Faculty"]=="exact_sci"]),
                      np.mean(data["Salary"][data["Faculty"]=="Humanities"])]
  sampled_meansCols = [np.mean(data["Salary"][data["Gender"]=="men"]),
                      np.mean(data["Salary"][data["Gender"]=="women"])]
  sampled_means = [np.mean(data["Salary"][(data["Faculty"]=="Social science")  & (data["Gender"]=="men")]),
                  np.mean(data["Salary"][(data["Faculty"]=="exact_sci")  & (data["Gender"]=="men")]),
                  np.mean(data["Salary"][(data["Faculty"]=="Humanities")  & (data["Gender"]=="men")]),
                  np.mean(data["Salary"][(data["Faculty"]=="Social science") & (data["Gender"]=="women")]),
                  np.mean(data["Salary"][(data["Faculty"]=="exact_sci")  & (data["Gender"]=="women")]),
                  np.mean(data["Salary"][(data["Faculty"]=="Humanities") & (data["Gender"]=="women")])]

  # Perform ANOVA and print table
  if show_me=="One-way anova for faculty":
    formula = 'Salary ~ Faculty'
    model = sm.formula.ols(formula, data=groupsDF).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    new_index=list(aov_table.index)
    new_index[1] = "Within groups"
    aov_table.index = new_index
    print(np.round(aov_table,3),"\n\n")
    print("Mean of all groups together (X) = ", round(sampled_Tmean,3))
    print("")
    print(tabulate([['Social science (X_1.)', sampled_means[0],sampled_means[3],sampled_meansRows[0]],
                    ['exact_sci (X_2.)', sampled_means[1],sampled_means[4],sampled_meansRows[1]],
                    ['Humanities',sampled_means[2],sampled_means[5],sampled_meansRows[2]],
                    ['Mean', sampled_meansCols[0],sampled_meansCols[1],sampled_Tmean] ],
                   headers=['Faculty\Gender', 'men ( X_.1 )', "women ( X_.2 )", "Mean"]))
 

  if show_me=="One-way anova for gender":
    formula = 'Salary ~ Gender'
    model = sm.formula.ols(formula, data=groupsDF).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    new_index=list(aov_table.index)
    new_index[1] = "Within groups"
    aov_table.index = new_index
    print(np.round(aov_table,3),"\n\n")
    print("Mean of all groups together (X) = ", round(sampled_Tmean,3))
    print("")
    print(tabulate([['Social science (X_1.)', sampled_means[0],sampled_means[3],sampled_meansRows[0]],
                    ['exact_sci (X_2.)', sampled_means[1],sampled_means[4],sampled_meansRows[1]],
                    ['Humanities',sampled_means[2],sampled_means[5],sampled_meansRows[2]],
                    ['Mean', sampled_meansCols[0],sampled_meansCols[1],sampled_Tmean] ],
                   headers=['Faculty\Gender', 'men ( X_.1 )', "women ( X_.2 )", "Mean"]))
 

  if show_me=="Two-way anova for gender and facutly":
    formula = 'Salary ~ Gender + Faculty + Gender:Faculty'
    model = sm.formula.ols(formula, data=groupsDF).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    new_index=list(aov_table.index)
    new_index[3] = "Within groups"
    aov_table.index = new_index
    print(np.round(aov_table,3),"\n\n")
    print("Mean of all groups together (X) = ", round(sampled_Tmean,3))
    print("")
    print(tabulate([['Social science (X_1.)', sampled_means[0],sampled_means[3],sampled_meansRows[0]],
                    ['exact_sci (X_2.)', sampled_means[1],sampled_means[4],sampled_meansRows[1]],
                    ['Humanities',sampled_means[2],sampled_means[5],sampled_meansRows[2]],
                    ['Mean', sampled_meansCols[0],sampled_meansCols[1],sampled_Tmean] ],
                   headers=['Faculty\Gender', 'men ( X_.1 )', "women ( X_.2 )", "Mean"]))
 
  if show_me=="t-test between men and women":
    results = perform_t_test(groupsDF,"Salary","Gender","men,women")
  if show_me=="t-test between social and exact sciences":
    results = perform_t_test(groupsDF,"Salary","Faculty","Social science,exact_sci")
  if show_me=="t-test between humanities and exact sciences":
    results = perform_t_test(groupsDF,"Salary","Faculty","Humanities,exact_sci")
  if show_me=="t-test between humanities and social sciences":
    results = perform_t_test(groupsDF,"Salary","Faculty","Humanities,Social science")
  if show_me=="Chi-square for independence of gender from faculty":
    results=show_contingency_table_chi2(groupsDF,"","Faculty","Gender")
  if show_me=="Summary statistics of salary by gender":
    results=print_means_sd_by_factor(groupsDF,"Gender","Salary")
  if show_me=="Summary statistics of salary by faculty":
    results=print_means_sd_by_factor(groupsDF,"Faculty","Salary")  

**<font size="6" color='cornflowerblue'><ins>Part 1</ins></font>**

In the first part, you will be asked to study six research questions. The following contains data on soccer players from different leagues in Europe (England, Germany, France, and Spain) who played in the 2012-2013 season. Run the block below to see the data and your personalized questions. The possible variables are listed below.

**The Variables (Assume that any categorical variable may influence any numeric variable):**


 * player – Each player's ID

 * leagueCountry – The country/league each player plays in. Either 'Spain', 'Germany', 'France' or 'England'.

 * height – Each player's height in centimeters.

 * weight – Each player's weight in kilograms.

 * position – The role of each player in the field. Either 'Attackers', 'Midfielders', 'Defenders' or 'Goalkeepers'.

 * games – The number of games each player played in the relevant season.

 * goals - the number of goals scored by each player.

 * yellowCards – The number of yellow cards each player got in the relevant season.

**Note** - these variable names are case sensitive. When you write the names of the variables be sure to write it exactly as they appear here.


Block 2#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>The data</ins>** 

block_id = 2
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block2():
  from IPython.display import display, HTML
  df = pd.read_csv("https://raw.githubusercontent.com/CogGal/CogGal/master/Dataset_final_proj.csv")
  position_dict = {"Attacking Midfielder":'Attacker' , "Center Forward":'Attacker' ,
                  "Left Winger":'Attacker' , "Right Winger":'Attacker',
                  "Center Midfielder":"Midfielder", "Defensive Midfielder":"Midfielder" ,
                  "Left Midfielder":"Midfielder", "Right Midfielder":"Midfielder",
                  "Center Back":"Defender", "Right Fullback":"Defender", "Left Fullback":"Defender",
                  "Goalkeeper":"Goalkeeper"}

  np.random.seed(group_number)
  size=np.random.choice([312,316,320,280,300,260])
  df=df[df["position"].isin(position_dict.keys())]
  for key in position_dict.keys():
    df.loc[df["position"] == key, "position"] = position_dict[key]
  leagueCountry = df["leagueCountry"].unique()
  n_lig = len(df["leagueCountry"].unique())

  sample=[]
  np.random.seed(group_number)
  scenario= np.random.choice([1,2,3])
  for count in range(n_lig):
    lig_i = count
    curr_league = leagueCountry[lig_i]
    if scenario==1:
      if leagueCountry[lig_i]=="France":
        curr_league="Spain"
        curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
        curr_sample["leagueCountry"]="France"
      elif leagueCountry[lig_i]=="Germany":
        curr_league="England"
        curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
        curr_sample["leagueCountry"]="Germany"
      else:
        curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
    if scenario==2:
      if leagueCountry[lig_i]=="France":
        curr_league="Spain"
        curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
        curr_sample["leagueCountry"]="France"
      else:
        curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
    if scenario==3:
      curr_sample = df[df["leagueCountry"] == curr_league].sample(int(size/n_lig),random_state=group_number+count)
    sample.append(curr_sample) 

  sample_df = pd.concat(sample)
  sample_df["player"]= np.arange(1,len(sample_df["player"])+1)
  sample_df=sample_df[["player","leagueCountry","height","weight","position","games","goals","yellowCards"]]
  questions={"basics": 
  ["Is there a height difference between players in the different leagues?"
  ,"Is there a weight difference between players in the different leagues?"
  ,"Is there a height difference between players in the different positions?"
  ,"Is there a weight difference between players in the different positions?"
  ,"Is there a difference in the number of games players played between the different positions?"
  ,"Is there a difference in the number of yellow cards players got between the different leagues?"
  ,"Is there a difference in the number of goals players scored between the different leagues?"
  ,"Is there a difference in the number of yellow cards players got between the different positions?"
  ,"Is there a difference in the number of goals players scored between the different positions?"],
  "interactions":[
  "Does the height difference between positions change between different leagues?"
  ,"Does the weight difference between positions change between different leagues?"
  ,"Does the difference in the number of games played between the positions change between different leagues?"
  ,"Does the difference in yellow cards players got in each league changes between the different positions?"
  ,"Does the difference in the number of goals players scored in each league changes between the different positions?"],
  "regression":
  ["Does the number of yellow cards change in a specific direction when number of games a player played grows? \nRegardless of the previous question, how would you model a linear relation between the variables?",
  "Does the number of goals a player scored change in a specific direction when number of games a player played grows? \nRegardless of the previous question, how would you model a linear relation between the variables?",
   "Does the number of yellow cards change in a specific direction when number of goals a player scored grows? \nRegardless of the previous question, how would you model a linear relation between the variables?",
   "Does the number of games a player played change in a specific direction when his height grows? \nRegardless of the previous question, how would you model a linear relation between the variables?",
   "Does the number of games a player played change in a specific direction when his weight grows? \nRegardless of the previous question, how would you model a linear relation between the variables?",
   "Does the player's weight played change in a specific direction when his height grows? \nRegardless of the previous question, how would you model a linear relation between the variables?"],
  "chisquared":[
  "Does the number of players in each position depend on the league they play in?"
  ,"Are the players equally distributed between all positions?"],
  "distribution":[
  "Is the number of yellow cards (roughly) normally distributed?"
  ,"Is the number of goals (roughly) normally distributed?"
  ,"Is the distribution of the number of games (roughly) skewed?"
  ,"Is the distribution of number of goals (roughly) skewed?"],
  "aparametric":[
  "Create the bootstrap distribution of the statistic used to test question A. What is the 95% confidence interval according to this distribution?"
  ,"Create the bootstrap distribution of the statistic used to test question B. What is the 95% confidence interval according to this distribution?"
  ,"Create the bootstrap distribution of the slope of the model predicting goals by height. What is the 95% confidence interval according to this distribution?"
  ,"Create the permutation H0 distribution of the statistic used to test question A. Is it significant at alpha=0.05?"
  ,"Create the permutation H0 distribution of the statistic used to test question B. Is it significant at alpha=0.05?"
  ,"Create the permutation based H0 distribution of the slope of the model predicting goals by height. Is it significantly different than zero at alpha=0.05?"]}
  
  np.random.seed(group_number)
  q12=np.random.choice([*questions['basics'],*questions['interactions']],2,replace=False)
  q3=np.random.choice(questions['chisquared'],1,replace=False)
  q4=np.random.choice(questions['regression'],1,replace=False)
  q5=(np.random.choice(questions['distribution'],1,replace=False))
  q6=(np.random.choice(questions['aparametric'],1,replace=False))

  if (q6==questions["aparametric"][0])|(q6==questions["aparametric"][1])|(q6==questions["aparametric"][2]):
    boot_perm_question_type = 'boot'
  else:
    boot_perm_question_type = 'perm'

  questions_for_group = [*q12,*q3,*q4,*q5,*q6]
  samp_df_print = sample_df.copy()

  samp_df_print.index=np.arange(1,len(samp_df_print["player"])+1)
  data['boot_perm_question_type'] = boot_perm_question_type
  data['questions_for_group'] = questions_for_group
  return samp_df_print, size, sample_df, questions_for_group, questions, boot_perm_question_type


try:
  prepare_block()
  samp_df_print, size, sample_df, questions_for_group, questions, boot_perm_question_type = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

samp_df_print

Block 3#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>The research questions</ins>** 

block_id = 3
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block3():
  question_label = ['A','B','C','D','E','F']
  increase_font()
  for i in range(6):
    print(f"Question {question_label[i]}: {questions_for_group[i]}\n")


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 4#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>If you want to learn more about the data...</ins>** 
#@markdown #**Choose a categorical and numerical variable**
#@markdown Note: This block is not graded, it is only here to assist you in the project.


block_id = 4
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block4():
  categorical = "Set variable" #@param ["Set variable","leagueCountry","position"]
  numeric = "Set variable" #@param ["Set variable","height","weight","goals","games","yellowCards"]
  print_means_sd_by_factor(sample_df, categorical, numeric)


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 5#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question A</ins>** 
#@markdown #**Run to display a reminder**

block_id = 5
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block5():
  increase_font()
  print(questions_for_group[0])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 6#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1.1</ins>** 
#@markdown #*Warning: Running this block will delete your answer to question 1.2* (And then you'll have to answer it again)
#@markdown #**Choose the appropriate test and variables to test Question A**
#@markdown Write the names of the variables exactly as they appear in the description above.
#@markdown If there are several possible tests, choose the one that utilizes the most data, e.g. the one that might convert unexplained variance into explained variance. 
#@markdown * For conducting t-test and one-way **ANOVA**, set independent variable 1 and the dependent variable. In two-way anova, set all 3 variables required.
#@markdown * In case of **linear regression/correlation**, set the predictor (x) as independent variable 1 and the predicted (y) as the dependent variable.
#@markdown * For conducting a **t-test**, set also the levels you wish to test between with a comma between them (e.g. "France,Spain")
#@markdown * For conducting a **Chi-squared test**, use the indenpendent variables needed (either just the first one or both of them, depends on the test)

block_id = 6
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block6():
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Chi squared of independence","2 sample t-test","Two-way ANOVA","One-way ANOVA","Linear regression","Chi squared of goodnes of fit","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable = "" #@param {type:"string"}
  levels_t_test = "" #@param {type:"string"}
  if test_func == perform_t_test:
    res,stat,df = perform_t_test(sample_df,dependent_variable,independent_variable1,levels_t_test)
  elif test_func == perform_anova_twoway:
    if questions_for_group[0][0:8]=='Does the':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[0][-10:-1]=='positions':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)
  answers[block_id] = (test, independent_variable1, independent_variable2, dependent_variable, levels_t_test)
  print('\n')
  print("Your answer:", answers[block_id])

  executed_blocks.discard(block_id + 1)
  # res.to_csv('res1')
  data[block_id] = stat, df


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 7#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1.2</ins>** 

block_id = 7
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block7():
  warnings.simplefilter(action='ignore', category=RuntimeWarning)
  #@markdown **Define which H0 distribution you would like to use**
  distribution = "Choose a distribution" #@param ["Choose a distribution","F","t","Chi_squared"]
  #@markdown **Degrees of freedom:** Set the values of the degrees of freedom in your test. If only one value is needed, set the second value to 0.
  deg_freedom1 =  0#@param {type:"number"}
  deg_freedom2 =  0#@param {type:"number"}
  #@markdown **Your result** :
  statistic =  0#@param {type:"number"}
  if distribution == "Choose a distribution":
    raise Exception("You must choose a distribution")
  fun=eval(distribution+"_fun")
  fun(deg_freedom1, deg_freedom2, statistic)

  answers[block_id] = distribution, deg_freedom1, deg_freedom2, statistic
  print("\nYour Answer is: ", answers[block_id], end = '\n\n')



try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 8#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question B</ins>** 
#@markdown #**Run to display a reminder**

block_id = 8
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block8():
  increase_font()
  print(questions_for_group[1])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 9#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 2.1</ins>** 
#@markdown #*Warning: Running this block will delete your answer to question 2.2* (And then you'll have to answer it again)
#@markdown #**Choose the appropriate test and variables to test Question B**
#@markdown Write the names of the variables exactly as they appear in the description above.
#@markdown If there are several possible tests, choose the one that utilizes the most data, e.g. the one that might convert unexplained variance into explained variance. 
#@markdown * For conducting t-test and one-way **ANOVA**, set independent variable 1 and the dependent variable. In two-way anova, set all 3 variables required.
#@markdown * In case of **linear regression/correlation**, set the predictor (x) as independent variable 1 and the predicted (y) as the dependent variable.
#@markdown * For conducting a **t-test**, set also the levels you wish to test between with a comma between them (e.g. "France,Spain")
#@markdown * For conducting a **Chi-squared test**, use the indenpendent variables needed (either just the first one or both of them, depends on the test)

block_id = 9
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block9():
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Chi squared of independence","2 sample t-test","Two-way ANOVA","One-way ANOVA","Linear regression","Chi squared of goodnes of fit","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable = "" #@param {type:"string"}
  levels_t_test = "" #@param {type:"string"}
  if test_func == perform_t_test:
    res,stat,df = perform_t_test(sample_df,dependent_variable,independent_variable1,levels_t_test)
  elif test_func == perform_anova_twoway:
    if questions_for_group[1][0:8]=='Does the':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[1][-10:-1]=='positions':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)
  
  executed_blocks.discard(block_id + 1)
  answers[block_id] = test, independent_variable1, independent_variable2, dependent_variable, levels_t_test
  # res.to_csv('res2')
  data[block_id] = stat, df
try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 10#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 2.2</ins>** 

block_id = 10
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block10():
  warnings.simplefilter(action='ignore', category=RuntimeWarning)
  #@markdown **Define which H0 distribution you would like to use**
  distribution = "Choose a distribution" #@param ["Choose a distribution","F","t","Chi_squared"]
  #@markdown **Degrees of freedom:** Set the values of the degrees of freedom in your test. If only one value is needed, set the second value to 0.
  deg_freedom1 =  0#@param {type:"number"}
  deg_freedom2 =  0#@param {type:"number"}
  #@markdown **Your result** :
  statistic =  0#@param {type:"number"}
  if distribution == "Choose a distribution":
    raise Exception("You must choose a distribution")
  fun=eval(distribution+"_fun")
  fun(deg_freedom1, deg_freedom2, statistic)

  answers[block_id] = distribution, deg_freedom1, deg_freedom2, statistic
  print("\nYour Answer is: ", answers[block_id], end = '\n\n')


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 11#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question C</ins>** 
#@markdown #**Run to display a reminder**

block_id = 11
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block11():
  increase_font()
  print(questions_for_group[2])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 12#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 3.1</ins>** 
#@markdown #*Warning: Running this block will delete your answer to question 3.2* (And then you'll have to answer it again)
#@markdown #**Choose the appropriate test and variables to test Question C**


block_id = 12
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block12():
#@markdown Write the names of the variables exactly as they appear in the description above.
#@markdown If there are several possible tests, choose the one that utilizes the most data, e.g. the one that might convert unexplained variance into explained variance. 
#@markdown * For conducting t-test and one-way **ANOVA**, set independent variable 1 and the dependent variable. In two-way anova, set all 3 variables required.
#@markdown * In case of **linear regression/correlation**, set the predictor (x) as independent variable 1 and the predicted (y) as the dependent variable.
#@markdown * For conducting a **t-test**, set also the levels you wish to test between with a comma between them (e.g. "France,Spain")
#@markdown * For conducting a **Chi-squared test**, use the indenpendent variables needed (either just the first one or both of them, depends on the test)
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Chi squared of independence","2 sample t-test","Two-way ANOVA","One-way ANOVA","Linear regression","Chi squared of goodnes of fit","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable = "" #@param {type:"string"}
  levels_t_test = "" #@param {type:"string"}
  if test_func == perform_t_test:
    res,stat,df = perform_t_test(sample_df,dependent_variable,independent_variable1,levels_t_test)
  elif test_func == perform_anova_twoway:
    if questions_for_group[0][0:8]=='Does the':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[0][-10:-1]=='positions':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)
  answers[block_id] = (test, independent_variable1, independent_variable2, dependent_variable, levels_t_test)
  
  executed_blocks.discard(block_id + 1)
  if isinstance(res, (float, int, list)):
    data[block_id] = stat, df, res
  else:
    # res.to_csv('res3')
    data[block_id] = stat, df

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 13#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 3.2</ins>** 

block_id = 13
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block13():
  warnings.simplefilter(action='ignore', category=RuntimeWarning)
  #@markdown **Define which H0 distribution you would like to use**
  distribution = "Choose a distribution" #@param ["Choose a distribution","F","t","Chi_squared"]
  #@markdown **Degrees of freedom:** Set the values of the degrees of freedom in your test. If only one value is needed, set the second value to 0.
  deg_freedom1 =  0#@param {type:"number"}
  deg_freedom2 =  0#@param {type:"number"}
  #@markdown **Your result** :
  statistic =  0#@param {type:"number"}
  if distribution == "Choose a distribution":
    raise Exception("You must choose a distribution")
  fun = eval(distribution+"_fun")
  fun(deg_freedom1, deg_freedom2, statistic)

  answers[block_id] = distribution, deg_freedom1, deg_freedom2, statistic
  print("\nYour Answer is: ", answers[block_id], end = '\n\n')


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 14#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question D</ins>** 
#@markdown #**Run to display a reminder**

block_id = 14
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block14():
  increase_font()
  print("Part 1:")
  print(questions_for_group[3].split("\n")[0])
  print("\nPart 2:")
  print(questions_for_group[3].split("\n")[1])

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 15#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4.1</ins>** 
#@markdown #*Warning: Running this block will delete your answer to question 4.2* (And then you'll have to answer it again)
#@markdown #**Choose the appropriate test and variables to test Question D - part 1**


block_id = 15
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block15():
  #@markdown Write the names of the variables exactly as they appear in the description above.
#@markdown If there are several possible tests, choose the one that utilizes the most data, e.g. the one that might convert unexplained variance into explained variance. 
#@markdown * For conducting t-test and one-way **ANOVA**, set independent variable 1 and the dependent variable. In two-way anova, set all 3 variables required.
#@markdown * In case of **linear regression/correlation**, set the predictor (x) as independent variable 1 and the predicted (y) as the dependent variable.
#@markdown * For conducting a **t-test**, set also the levels you wish to test between with a comma between them (e.g. "France,Spain")
#@markdown * For conducting a **Chi-squared test**, use the indenpendent variables needed (either just the first one or both of them, depends on the test)
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Chi squared of independence","2 sample t-test","Two-way ANOVA","One-way ANOVA","Linear regression","Chi squared of goodnes of fit","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable = "" #@param {type:"string"}
  levels_t_test = "" #@param {type:"string"}
  if test_func == perform_t_test:
    res,stat,df = perform_t_test(sample_df,dependent_variable,independent_variable1,levels_t_test)
  elif test_func == perform_anova_twoway:
    if questions_for_group[0][0:8]=='Does the':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[0][-10:-1]=='positions':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)
 
  executed_blocks.discard(block_id + 1)
  answers[block_id] = test, independent_variable1, independent_variable2, dependent_variable, levels_t_test
  # res.to_csv('res4')
  data[block_id] = stat, df


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 16#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4.2</ins>** 

block_id = 16
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block16():
  warnings.simplefilter(action='ignore', category=RuntimeWarning)
  #@markdown **Define which H0 distribution you would like to use to test question D-part 1**
  distribution = "Choose a distribution" #@param ["Choose a distribution","F","t","Chi_squared"]
  #@markdown **Degrees of freedom:** Set the values of the degrees of freedom in your test. If only one value is needed, set the second value to 0.
  deg_freedom1 =  0#@param {type:"number"}
  deg_freedom2 = 0 #@param {type:"number"}
  #@markdown **Your result**:
  statistic =  0#@param {type:"number"}
  if distribution == "Choose a distribution":
    raise Exception("You must choose a distribution")
  fun=eval(distribution+"_fun")
  fun(deg_freedom1, deg_freedom2, statistic)
  print("\n")

  answers[block_id] = distribution, deg_freedom1, deg_freedom2, statistic
  print("\nYour Answer is: ", answers[block_id], end = '\n\n')



try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 17#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4.3</ins>** 
#@markdown #*Warning: Running this block will delete your answer to question 4.4* (And then you'll have to answer it again)
#@markdown #**How would you model a linear relationship between the variables? (Question D, part 2)**
#@markdown Choose the appropriate test and variables to model the linear relationship.

#@markdown Write the names of the variables exactly as they appear in the description above.
#@markdown * Set only variables that you need for your specific test in order to minimize the unexplained variance. If you don't need two independent variables, set only the first one. 
#@markdown * In case of **linear regression/correlation**, set the predictor (x) as independent variable 1 and the predicted (y) as the dependent variable.
#@markdown * For conducting a **t-test**, set also the levels you wish to test between with a comma between them (e.g. "France,Spain")
#@markdown * For conducting a **Chi-squared test**, use the indenpendent variables

block_id = 17
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block17():
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Chi squared of independence","2 sample t-test","Two-way ANOVA","One-way ANOVA","Linear regression","Chi squared of goodnes of fit","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable = "" #@param {type:"string"}
  levels_t_test = "" #@param {type:"string"}
  if test_func == perform_t_test:
    res,stat,df = perform_t_test(sample_df,dependent_variable,independent_variable1,levels_t_test)
  elif test_func == perform_anova_twoway:
    if questions_for_group[0][0:8]=='Does the':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[0][-10:-1]=='positions':
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    res,stat,df = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)
  answers[block_id] = test, independent_variable1, independent_variable2, dependent_variable, levels_t_test
  
  executed_blocks.discard(block_id + 1)
  if isinstance(res, np.ndarray):
    data['res'] = res.tolist()
  elif isinstance(res, list):
    data['res'] = [x.tolist() if isinstance(x, np.ndarray) else x for x in res]
  else:
    increase_font()
    print("An error occured. Please contact course staff and send them this colab.")
  data[block_id] = stat, df

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 18#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4.4</ins>** 
#@markdown #**What is the fraction of the explained variance this model has?**
#@markdown Note: If it's lower than 0.0001, just write zero.
#Avner - pre-set this to 1.
block_id = 18
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block18():
  Answer = 1 #@param {type:"number"}
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 19#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question E</ins>** 
#@markdown #**Run to display a reminder**

block_id = 19
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block19():
  increase_font()
  print(questions_for_group[4])

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 20#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5</ins>** 
#@markdown #**Answer research question E (Use the analysis tool you have at the beginning of this notebook)**

block_id = 20
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block20():
  Answer = "Choose an answer" #@param ["Choose an answer","Yes","No"]
  if Answer == "Choose an answer":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 21#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research question F</ins>** 
#@markdown #**Run to display a reminder**

block_id = 21
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block21():
  increase_font()
  print(questions_for_group[5])

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 22#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6</ins>** 
#@markdown #**Choose the analysis to perform, how many times to perform it, and on which dependent and independent variable**
#@markdown * Set the independent and dependent variables **exactly** by the instructions that appears in the 4 questions before which included this analysis

#@markdown * Think whether you want to sample with or without replacement and whether you need to shuffle the lables of the independent variables (shuffle_groups)
#@markdown * In case of two-way anova, also choose the statistic you want, the main effect of IV1/IV2 or the interaction

#@markdown * If you choose repetitions = 5000 then running this block will take a while, and will require patience.
block_id = 22
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block22():
  func_dict = {"Two-way ANOVA":perform_anova_twoway,"2 sample t-test":perform_t_test,"Chi squared of independence":show_contingency_table_chi2,"Chi squared of goodnes of fit":show_frequency_table_chi2,"One-way ANOVA":perform_anova_oneway,"Linear regression":perform_linear_regression,"Pearson correlation":perform_cor_test,"Spearman correlation":perform_spearman_test}
  test = "Choose a test" #@param ["Choose a test","Two-way ANOVA","One-way ANOVA","Linear regression","Pearson correlation","Spearman correlation"] 
  test_func = func_dict[test]
  independent_variable1 = "" #@param {type:"string"}
  independent_variable2 = "" #@param {type:"string"}
  dependent_variable =  "" #@param {type:"string"}
  print("Performing original analysis on the original data sample without sampling/shuffling:")
  if test_func == perform_anova_twoway:
    if questions_for_group[0][0:8]=='Does the':
      _ = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=2)
    elif questions_for_group[0][-10:-1]=='positions':
      _ = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=0)
    else:
      _ = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2,effect=1)
  else:
    _ = test_func(sample_df,dependent_variable,independent_variable1,independent_variable2)

  #@markdown * Sampling settings:
  repetitions = "1" #@param [1,5,50,300,5000]
  shuffle_groups=False #@param {type:"boolean"}
  sample_with_replacement=False #@param {type:"boolean"}
  repetitions = eval(repetitions)
  effect_in_two_way_anova = "Set only in two-way ANOVA" #@param ["Set only in two-way ANOVA","Interaction","Factor 1","Factor 2"]

  print(f"\n\nPerforming the requested sampling with {repetitions} repetitions:")
  stat_distribution=create_dist_by_choice(data=sample_df,var1=independent_variable1,function=test_func,
                          repetitions=repetitions,replacement=sample_with_replacement,
                          shuffling_var1=shuffle_groups,shuffling_var2=shuffle_groups,
                          var2=independent_variable2,dv=dependent_variable,effect=effect_in_two_way_anova)

  answers[block_id] = test, independent_variable1, independent_variable2, dependent_variable, repetitions, shuffle_groups, sample_with_replacement, effect_in_two_way_anova
  print("Your answer:", answers[block_id])

  return stat_distribution


try:
  prepare_block()
  stat_distribution = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 23#

In [ ]:
#@markdown #**Use the following interface to get the values you need for the question above**
#@markdown #*Warning: Running this block will delete your answer to question 7* (And then you'll have to answer it again)

block_id = 23
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block23():
  
  #@markdown To get the number of observations above your statistic 
  #@markdown **Note: You may need the number below it, depending on your hypothesis**
  statistic = 0 #@param {type:"number"}
  #@markdown To get a quantile of the distribution (e.g. the median, the 25% quantile, the 75% quantile, or any other value in the distribution below which we have x% of the observations...). Enter a number between 0-1:
  quantile = 0 #@param {type:"number"}
  CI = tuple(np.round(np.quantile(stat_distribution,[0.025,0.975]),2))
  one_tail_p = np.mean(stat_distribution>statistic)
  if (statistic<0):
    one_tail_p = np.mean(stat_distribution<statistic)

  print(f"Observations above {statistic} in the distribution: {np.sum(statistic<stat_distribution)}")
  print(f"Quantile {100*quantile}: {np.round(np.quantile(stat_distribution,quantile),2)}")
  
  executed_blocks.discard(block_id + 1)
  data['CI'] = CI
  data['one_tail_p'] = one_tail_p
  # answers[block_id] = statistic, quantile
  # print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 24#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7</ins>** 
#@markdown #**Run to see question**
#@markdown Note: Then run again to save your answer

block_id = 24
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block24():
  question_bootperm_dict = {"perm":"What is the (one-tailed) p-value from the permutation test? Note that if you got a negative statistic, the tool above returns the number of values greater than your value, and you need the complement of this number.",
                            "boot":"What is the 95% confidence interval for your statistic? Write it with a comma between, and the smaller value to the left. For example - '-1.96,1.87'"}
  question_bootperm_text = question_bootperm_dict[boot_perm_question_type]
  print(question_bootperm_text + '\n')

  Answer = "0" #@param {type:"string"}
  Answer = eval(Answer)
  increase_font()

  # Checking if this is the first time this block was run
  global q7_flag
  if not q7_flag:
    if boot_perm_question_type == "boot":
      print("Set two values seperated by a comma")
    elif boot_perm_question_type == "perm":
      print("Enter a numerical value")
    else:
      print("Question type not recognized. Please contact the course staff.")
    q7_flag = True
    return

  if boot_perm_question_type == "boot":
    if not isinstance(Answer, tuple)or len(Answer) != 2:
      raise Exception("You must set two values seperated by a comma")
  elif boot_perm_question_type == "perm":
    if not isinstance(Answer, (int, float)):
      raise Exception("You must enter a numerical value")
  else:
    raise Exception("Question type not recognized. Please contact the course staff.")

  executed_blocks.add(block_id)
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  
except Exception as e:
  print(str(e))


**<font size="6" color='cornflowerblue'><ins>Part 2</ins></font>**

#Each of the following questions is composed of a general research description, the conclusion drawn from the research, and an interactive block that allows you to investigate the data and conduct various analyses. Afterward, a question will be presented.

#Please note that although some of the questions are similar, they are all independent from one another. Your answers should only consider the information given in that specific scenario. Also, inspect your data and the given descriptions with care, changes between scenarios are critical.

Block 25#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario A</ins>** 

block_id = 25
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block25():
  Q_fun = eval(f"Q{Qlist[0][0]}")
  question_1_output = Q_fun(Qlist[0][1])
  increase_font()
  print(question_1_output[1])
  print(f"\nConclusion: {question_1_output[2]}")
  return question_1_output


try:
  prepare_block()
  question_1_output = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 26#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Analysis tools</ins>** 
#@markdown Run this block to use one (or more) of the following tools to check if, given the presented information, the conclusion above is the most probable conclusion.
#@markdown * There is no limitation on running this block and it is not graded.

block_id = 26
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block26():
  options = ["Choose a tool","Histogram of serotonin consumption",
            "Histogram of depression level",
            "t-test between serotonin consumption and depression level",
            "Pearson correlation between serotonin consumption and depression level",
            "Spearman correlation between serotonin consumption and depression level",
            "Linear regression between serotonin consumption and depression level",
            "Bootstrap distribution of regression slope"] 
  data_q1 = pd.DataFrame({"Serotonin consumption":question_1_output[0][0],
                        "Depression level":question_1_output[0][1]})
  data_q1.to_csv('data_q1')
  show_me = "Choose a tool" #@param ["Choose a tool","Histogram of serotonin consumption", "Histogram of depression level","paired t-test between serotonin consumption and depression level", "Pearson correlation between serotonin consumption and depression level", "Spearman correlation between serotonin consumption and depression level", "Linear regression between serotonin consumption and depression level", "Bootstrap distribution of regression slope"] 

  if show_me=="Histogram of depression level":
    show_histogram(data_q1,"Depression level")
  if show_me=="Histogram of serotonin consumption":
    show_histogram(data_q1,"Serotonin consumption")
  if show_me=="paired t-test between serotonin consumption and depression level":
    results=stats.ttest_rel(data_q1["Serotonin consumption"],data_q1["Depression level"])
    print(f"t(49) = {np.round(results[0],3)}, p = {np.round(results[1],12)}")
  if show_me=="Pearson correlation between serotonin consumption and depression level":
    results=stats.pearsonr(data_q1["Serotonin consumption"],data_q1["Depression level"])
    print(f"r(49) = {np.round(results[0],3)}, p = {np.round(results[1],8)}")
  if show_me=="Spearman correlation between serotonin consumption and depression level":
    results=stats.spearmanr(data_q1["Serotonin consumption"],data_q1["Depression level"])
    print(f"r(49) = {np.round(results[0],3)}, p = {np.round(results[1],8)}")
  if show_me=="Linear regression between serotonin consumption and depression level":
    _=perform_linear_regression(data_q1,"Depression level","Serotonin consumption")
    plt.figure(figsize=(7,5))
    sns.regplot(data=data_q1,x="Serotonin consumption",y="Depression level")
  if show_me=="Bootstrap distribution of regression slope":
    create_dist_by_choice(data=data_q1,var1="Serotonin consumption",function=perform_linear_regression,
                          repetitions=5000,replacement=True,
                          shuffling_var1=False,shuffling_var2=False,
                          var2="",dv="Depression level")


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 27#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 8</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, is the conclusion correct?**

block_id = 27
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block27():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 28#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario B</ins>** 

block_id = 28
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block28():

  Q_fun = eval(f"Q{Qlist[1][0]}")
  question_2_output = Q_fun(Qlist[1][1])
  increase_font()
  print(question_2_output[1])
  print(f"\nConclusion: {question_2_output[2]}")

  return question_2_output


try:
  prepare_block()
  question_2_output = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 29#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Analysis tools</ins>** 
#@markdown Run this block to use one (or more) of the following tools to check if, given the presented information, the conclusion above is the most probable conclusion.
#@markdown * There is no limitation on running this block and it is not graded.

block_id = 29
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block29():
  options = ["Choose a tool","Histogram of serotonin consumption",
            "Histogram of depression level",
            "t-test between serotonin consumption and depression level",
            "Pearson correlation between serotonin consumption and depression level",
            "Spearman correlation between serotonin consumption and depression level",
            "Linear regression between serotonin consumption and depression level",
            "Bootstrap distribution of regression slope"] 
  data_q2 = pd.DataFrame({"Serotonin consumption":question_2_output[0][0],
                        "Depression level":question_2_output[0][1]})
  data_q2.to_csv('data_q2')
  show_me = "Choose a tool" #@param ["Choose a tool","Histogram of serotonin consumption", "Histogram of depression level","paired t-test between serotonin consumption and depression level", "Pearson correlation between serotonin consumption and depression level", "Spearman correlation between serotonin consumption and depression level", "Linear regression between serotonin consumption and depression level", "Bootstrap distribution of regression slope"] 

  if show_me=="Histogram of depression level":
    show_histogram(data_q2,"Depression level")
  if show_me=="Histogram of serotonin consumption":
    show_histogram(data_q2,"Serotonin consumption")
  if show_me=="paired t-test between serotonin consumption and depression level":
    results=stats.ttest_rel(data_q2["Serotonin consumption"],data_q2["Depression level"])
    print(f"t(49) = {np.round(results[0],3)}, p = {np.round(results[1],12)}")
  if show_me=="Pearson correlation between serotonin consumption and depression level":
    results=stats.pearsonr(data_q2["Serotonin consumption"],data_q2["Depression level"])
    print(f"r(49) = {np.round(results[0],3)}, p = {np.round(results[1],8)}")
  if show_me=="Spearman correlation between serotonin consumption and depression level":
    results=stats.spearmanr(data_q2["Serotonin consumption"],data_q2["Depression level"])
    print(f"r(49) = {np.round(results[0],3)}, p = {np.round(results[1],8)}")
  if show_me=="Linear regression between serotonin consumption and depression level":
    _=perform_linear_regression(data_q2,"Depression level","Serotonin consumption")
    plt.figure(figsize=(7,5))
    sns.regplot(data=data_q2,x="Serotonin consumption",y="Depression level")
  if show_me=="Bootstrap distribution of regression slope":
    create_dist_by_choice(data=data_q2,var1="Serotonin consumption",function=perform_linear_regression,
                          repetitions=5000,replacement=True,
                          shuffling_var1=False,shuffling_var2=False,
                          var2="",dv="Depression level")
    




try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 30#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 9</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, is the conclusion correct?**

block_id = 30
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block30():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 31#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario C</ins>** 

block_id = 31
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block31():
  Q_fun = eval(f"Q{Qlist[2][0]}")
  question_3_output = Q_fun(Qlist[2][1])
  increase_font()
  print(question_3_output[1])
  print(f"\nConclusion: {question_3_output[2]}")

  return question_3_output


try:
  prepare_block()
  question_3_output = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 32#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Analysis tools</ins>** 
#@markdown Run this block to use one (or more) of the following tools to check if, given the presented information, the conclusion above is the most probable conclusion.
#@markdown * There is no limitation on running this block and it is not graded.

block_id = 32
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block32():
  options = ["Choose a tool","Histogram of new drug",
            "Histogram of placebo",
            "t-test between new drug and placebo",
            "Pearson correlation between new drug and placebo",
            "Linear regression between new drug and placebo",
            "Bootstrap distribution of the difference between means",
            "H0 permutation distribution of the difference between means"] 
  data_q3 = pd.DataFrame({"new drug":question_3_output[0][1],
                        "placebo":question_3_output[0][0]})
  data_q3.to_csv('data_q3')
  mean_diff = np.round(np.mean(data_q3["new drug"])-np.mean(data_q3["placebo"]),3)
  print(f"The actual difference between means is {mean_diff}")
  show_me = "Choose a tool" #@param ["Choose a tool","Histogram of new drug","Histogram of placebo","H0 permutation distribution of the difference between means","t-test between new drug and placebo","Pearson correlation between new drug and placebo","Linear regression between new drug and placebo","Bootstrap distribution of the difference between means"]
  if show_me=="Histogram of placebo":
    show_histogram(data_q3,"placebo")
  if show_me=="Histogram of new drug":
    show_histogram(data_q3,"new drug")
  if show_me=="t-test between new drug and placebo":
    results=stats.ttest_ind(data_q3["new drug"],data_q3["placebo"])
    print(f"t(49) = {np.round(results[0],3)}, p = {np.round(results[1],12)}")
  if show_me=="Pearson correlation between new drug and placebo":
    results=stats.pearsonr(data_q3["new drug"],data_q3["placebo"])
    print(f"r(49) = {np.round(results[0],3)}, p = {np.round(results[1],8)}")
  if show_me=="Bootstrap distribution of the difference between means":
    dist =  boot_dist_mean_diff(data_q3["new drug"],data_q3["placebo"])
    sns.displot(dist)
  if show_me=="H0 permutation distribution of the difference between means":
    dist =  perm_dist_mean_diff(data_q3["new drug"],data_q3["placebo"])
    sns.displot(dist)
  if show_me=="Linear regression between new drug and placebo":
    _=perform_linear_regression(data_q3,"placebo","new drug")
    plt.figure(figsize=(7,5))
    sns.regplot(data=data_q3,x="new drug",y="placebo")


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 33#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 10</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, is the conclusion correct?**

block_id = 33
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block33():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 34#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario D</ins>** 

block_id = 34
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block34():
  Q_fun = eval(f"Q{Qlist[3][0]}")
  question_4_output = Q_fun(Qlist[3][1])
  increase_font()
  print(question_4_output[1])
  print(f"\nConclusion: {question_4_output[2]}")

  return question_4_output


try:
  prepare_block()
  question_4_output = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 35#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Analysis tools</ins>** 
#@markdown Run this block to use one (or more) of the following tools to check if, given the presented information, the conclusion above is the most probable conclusion.
#@markdown * There is no limitation on running this block and it is not graded.

block_id = 35
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block35():
  options = ["Choose a tool","Histogram of number of people","Summary statistics of the number of people","Bootstrap distribution of the number of people"] 
  data_q4 = pd.DataFrame({"number of people":question_4_output[0]})
  data_q4.to_csv('data_q4')
  show_me = "Choose a tool" #@param ["Choose a tool","Histogram of number of people","Summary statistics of the number of people","Bootstrap distribution of the number of people"] 
  
  if show_me=="Summary statistics of the number of people":
    nppl=data_q4["number of people"]
    print(f"The mean is {nppl.mean()} with SD={np.round(nppl.std(),2)}, the median is {np.median(nppl)}")
  if show_me=="Histogram of number of people":
    show_histogram(data_q4,"number of people")
  if show_me=="Bootstrap distribution of the number of people":
    dist =  boot_dist(data_q4["number of people"])
    sns.displot(dist)


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 36#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 11</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, is the conclusion correct?**

block_id = 36
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block36():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 37#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario E + Analysis tools</ins>** 
#@markdown Note that the analysis tool here is in this block and reacting immediately to your response.

block_id = 37
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block37():
  Q_fun = eval(f"Q{Qlist[4][0]}")
  _=Q_fun(Qlist[4][1])
  print("Below is a graph showing the differences between the observed y value and the predicted y value (from the linear regression) as a function of the x variable values")


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 38#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 12</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, what is the answer to the question printed in the block above?**

block_id = 38
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block38():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 39#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Research scenario F + Analysis tools</ins>** 
#@markdown Note that the analysis tool here is in this block and reacting immediately to your response.

block_id = 39
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block39():
  Q_fun = eval(f"Q{Qlist[5][0]}")
  _=Q_fun(Qlist[5][1])



try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 40#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 13</ins>** 
#@markdown #**Given the presented information and the analysis tools you have above, what is the answer to the question printed in the block above?**

block_id = 40
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block40():
  Answer = "Choose an answer." #@param ["Choose an answer.","Yes","No"]
  if Answer == "Choose an answer.":
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

<br>
<br>
<hr>
<br>
<br>

**<p><font size="10"><font color='cornflowerblue'> Good job!</ins></p>** </font>

#**You finished answering the questions for this exercise! Here are some important things to consider:**


* You can view your answers by running the next block. 
*After changing an answer to a question, you will need to run the 'Show your answers' again for it to reflect changes.
* <b>If you change an answer to a question, you will need to run it again for changes to be saved. 


Block #41

In [ ]:
#@title **<font color = cornflowerblue> <font size = 6>Show Your Answers <br>(Run to refresh after saving a question)**

block_id = 41
required_blocks = range(block_id)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def answers_printer():
  for block in sorted(answers):
    print("Question " + str(blocks_mapping[block]) + ": " + str(answers[block]))

try:
  prepare_block()
  answers_printer()
except Exception as e:
  print(str(e))


Block #42

In [ ]:
#@title  **<p><font size="10"><font color='cornflowerblue'><ins> <---  💾 SAVE! </ins></p>**
#@markdown If you are ready to submit your answers, go ahead and press 'Save!'💾 . This will prompt you to download your answers file. After downloading you will need to submit the file to moodle. Good luck!

from google.colab import files
from shutil import make_archive
from os import path, remove
from zipfile import ZipFile

block_id = 42
required_blocks = range(block_id - 1)

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def answer_saver(file):
  for block in sorted(answers):
    file.write("Question " + str(blocks_mapping[block]) + ": " + str(answers[block]) + '\n')


def submission_saver():
  # Writing answers
  f = open(exercise_name + "_submission", "w")
  f.write("#"+str(id_number) + "\n")
  f.write("#"+str(group_number) + "\n")
  answer_saver(f) 
  f.write("#End of answers")
  f.close()

  # Writing data
  g = open(exercise_name + "_data", "w")
  data["Submission time"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
  g.write(str(data))
  g.close()

  # Creating submission file
  zipObj = ZipFile(exercise_name + '.zip', 'w')
  zipObj.write(exercise_name + '_submission')
  zipObj.write(exercise_name + "_data")
  zipObj.write("data_q1")
  zipObj.write("data_q2")
  zipObj.write("data_q3")
  zipObj.write("data_q4")
  zipObj.close()
  files.download(exercise_name + '.zip') 

  # Message to user
  print("Well done! You are now downloading your submission file")


try:
  prepare_block()
  submission_saver()
except Exception as e:
  print(str(e))

